In [3]:

!pip install pdfplumber pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.3 MB/s eta 0:00:00


In [4]:
# Cell 2: Import libraries
import pdfplumber
import pandas as pd
import re
import os


In [5]:

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set your file path
pdf_filename = '/content/drive/MyDrive/Soma/senior-schools-in-kenya.pdf'
print(f"File path set to: {pdf_filename}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File path set to: /content/drive/MyDrive/Soma/senior-schools-in-kenya.pdf
✅ File found!


This PDF extraction function implements a two-stage approach to handle different PDF formats reliably. Here's how it works:

Initial Setup & Page Processing: The function opens the PDF file, counts total pages, and processes each page sequentially while providing progress feedback to track extraction status.

Primary Method - Table Detection: It first attempts structured table extraction using page.extract_tables(), which works best for PDFs with properly formatted table structures. When tables are found, it validates each row by checking if the first column contains a number (indicating a valid school record), then maps the 12 expected columns (S/No through Gender) to a dictionary structure.
Fallback Method - Text Parsing: If no tables are detected, the function switches to raw text extraction using page.extract_text(). It splits the text into lines, identifies data rows using regex pattern ^\d+\s (lines starting with numbers), and splits these lines by multiple spaces or tabs to separate the columns.

Data Validation & Structure: Both methods create standardized dictionaries with consistent field names (S_No, REGION, COUNTY, SUB_COUNTY, UIC, KNEC, SCHOOL_NAME, CLUSTER, TYPE, DISABILITY_TYPE, ACCOMMODATION_TYPE, GENDER), ensuring uniform output regardless of extraction method.

Debugging & Error Handling: The function includes comprehensive logging that shows pages processed, tables found, total rows extracted, and sample data. If extraction fails completely, it displays the first 1000 characters of raw PDF content to help diagnose formatting issues.

Robust Column Handling: The code safely handles varying column counts using conditional checks (if len(parts) > X) and provides empty strings for missing columns, preventing index errors when PDFs have inconsistent formatting or missing data fields.

In [8]:

def extract_school_data(pdf_path):
    rows = []

    with pdfplumber.open(pdf_path) as pdf:
        print(f"PDF has {len(pdf.pages)} pages")

        for page_num, page in enumerate(pdf.pages):
            print(f"Processing page {page_num + 1}...")

            # Try table extraction first
            tables = page.extract_tables()
            if tables:
                print(f"Found {len(tables)} table(s) on page {page_num + 1}")
                for table in tables:
                    for row in table[1:]:  # Skip header
                        if row and len(row) >= 8 and row[0] and row[0].strip().isdigit():
                            data_row = {
                                'S_No': row[0].strip() if row[0] else '',
                                'REGION': row[1].strip() if len(row) > 1 and row[1] else '',
                                'COUNTY': row[2].strip() if len(row) > 2 and row[2] else '',
                                'SUB_COUNTY': row[3].strip() if len(row) > 3 and row[3] else '',
                                'UIC': row[4].strip() if len(row) > 4 and row[4] else '',
                                'KNEC': row[5].strip() if len(row) > 5 and row[5] else '',
                                'SCHOOL_NAME': row[6].strip() if len(row) > 6 and row[6] else '',
                                'CLUSTER': row[7].strip() if len(row) > 7 and row[7] else '',
                                'TYPE': row[8].strip() if len(row) > 8 and row[8] else '',
                                'DISABILITY_TYPE': row[9].strip() if len(row) > 9 and row[9] else '',
                                'ACCOMMODATION_TYPE': row[10].strip() if len(row) > 10 and row[10] else '',
                                'GENDER': row[11].strip() if len(row) > 11 and row[11] else ''
                            }
                            rows.append(data_row)

            # If no tables found, try text extraction
            if not tables:
                text = page.extract_text()
                if text:
                    lines = text.split('\n')
                    for line in lines:
                        line = line.strip()
                        # Look for lines starting with numbers
                        if re.match(r'^\d+\s', line):
                            # Split by multiple spaces or tabs
                            parts = re.split(r'\s{2,}|\t+', line)

                            if len(parts) >= 8:
                                data_row = {
                                    'S_No': parts[0].strip(),
                                    'REGION': parts[1].strip() if len(parts) > 1 else '',
                                    'COUNTY': parts[2].strip() if len(parts) > 2 else '',
                                    'SUB_COUNTY': parts[3].strip() if len(parts) > 3 else '',
                                    'UIC': parts[4].strip() if len(parts) > 4 else '',
                                    'KNEC': parts[5].strip() if len(parts) > 5 else '',
                                    'SCHOOL_NAME': parts[6].strip() if len(parts) > 6 else '',
                                    'CLUSTER': parts[7].strip() if len(parts) > 7 else '',
                                    'TYPE': parts[8].strip() if len(parts) > 8 else '',
                                    'DISABILITY_TYPE': parts[9].strip() if len(parts) > 9 else '',
                                    'ACCOMMODATION_TYPE': parts[10].strip() if len(parts) > 10 else '',
                                    'GENDER': parts[11].strip() if len(parts) > 11 else ''
                                }
                                rows.append(data_row)

    print(f"Extracted {len(rows)} rows total")
    return rows

# Extract data
school_data = extract_school_data(pdf_filename)

# Debug: Show first few raw extractions
if school_data:
    print("Sample extracted data:")
    for i, row in enumerate(school_data[:3]):
        print(f"Row {i+1}: {row}")
else:
    print("No data extracted. Let's debug the PDF content...")

    # Debug: Show raw text from first page
    with pdfplumber.open(pdf_filename) as pdf:
        first_page_text = pdf.pages[0].extract_text()
        print("First 1000 characters of page 1:")
        print(first_page_text[:1000])

PDF has 248 pages
Processing page 1...
Found 1 table(s) on page 1
Processing page 2...
Found 1 table(s) on page 2
Processing page 3...
Found 1 table(s) on page 3
Processing page 4...
Found 1 table(s) on page 4
Processing page 5...
Found 1 table(s) on page 5
Processing page 6...
Found 1 table(s) on page 6
Processing page 7...
Found 1 table(s) on page 7
Processing page 8...
Found 1 table(s) on page 8
Processing page 9...
Found 1 table(s) on page 9
Processing page 10...
Found 1 table(s) on page 10
Processing page 11...
Found 1 table(s) on page 11
Processing page 12...
Found 1 table(s) on page 12
Processing page 13...
Found 1 table(s) on page 13
Processing page 14...
Found 1 table(s) on page 14
Processing page 15...
Found 1 table(s) on page 15
Processing page 16...
Found 1 table(s) on page 16
Processing page 17...
Found 1 table(s) on page 17
Processing page 18...
Found 1 table(s) on page 18
Processing page 19...
Found 1 table(s) on page 19
Processing page 20...
Found 1 table(s) on page 20


In [9]:
# Cell 5: Create DataFrame and preview
df = pd.DataFrame(school_data)
print("Data preview:")
print(df.head())

Data preview:
  S_No       REGION   COUNTY       SUB_COUNTY   UIC      KNEC  \
0    1  RIFT VALLEY  BARINGO  BARINGO CENTRAL  7J98  33517209   
1    2  RIFT VALLEY  BARINGO  BARINGO CENTRAL  GMUY  33517103   
2    3  RIFT VALLEY  BARINGO  BARINGO CENTRAL  5XVV  33517112   
3    4  RIFT VALLEY  BARINGO  BARINGO CENTRAL  5PK2  33517110   
4    5  RIFT VALLEY  BARINGO  BARINGO CENTRAL  WDP3  33517223   

                           SCHOOL_NAME CLUSTER    TYPE DISABILITY_TYPE  \
0        A I C PHILEMON CHELAGAT GIRLS      C3  PUBLIC         REGULAR   
1      AIC KAPKELELWA SECONDARY SCHOOL      C3  PUBLIC         REGULAR   
2             BEKIBON SECONDARY SCHOOL      C4  PUBLIC         REGULAR   
3  CHEPKERO MIXED DAY SECONDARY SCHOOL      C4  PUBLIC         REGULAR   
4  CHESONGO MIXED DAY SECONDARY SCHOOL      C4  PUBLIC         REGULAR   

  ACCOMMODATION_TYPE    GENDER  
0               NONE  BOARDING  
1               NONE  BOARDING  
2               NONE       DAY  
3               NO

In [10]:

csv_filename = '/content/drive/MyDrive/Soma/senior-schools-kenya.csv'
df.to_csv(csv_filename, index=False)
print(f" Saved to Google Drive: {csv_filename}")

# Also save a local copy for download
local_csv = 'senior-schools-kenya.csv'
df.to_csv(local_csv, index=False)

# Download the CSV file
from google.colab import files
files.download(local_csv)
print("📥 Downloaded local copy")

# Cell 8: Display summary statistics
print("\nDataset Summary:")
print(f"Total schools: {len(df)}")
print(f"Regions: {df['REGION'].nunique()}")
print(f"Counties: {df['COUNTY'].nunique()}")
print("\nSchools by Region:")
print(df['REGION'].value_counts())

 Saved to Google Drive: /content/drive/MyDrive/Soma/senior-schools-kenya.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Downloaded local copy

Dataset Summary:
Total schools: 9075
Regions: 8
Counties: 48

Schools by Region:
REGION
RIFT VALLEY      2580
EASTERN          1964
NYANZA           1566
CENTRAL          1108
WESTERN          1083
COAST             486
NORTH EASTERN     180
NAIROBI           108
Name: count, dtype: int64
